<a href="https://colab.research.google.com/github/chaitanyamanem/AutomaticProfileSelection/blob/main/Copy_of_RLHF_Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This models is with T5 paraphraser


In [5]:
%pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [6]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [7]:
%pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 10.5 MB/s eta 0:00:00


In [8]:
import random
import torch
import wandb
import time
import os
from tqdm.notebook import  tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt
import math

#tqdm.pandas()

from datasets import load_dataset

from transformers import AutoTokenizer, pipeline

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, AutoModelForSeq2SeqLMWithValueHead, create_reference_model

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [9]:
sentiment_pipe_kwargs = {"top_k": None, "function_to_apply": "none"}

config = PPOConfig(
    model_name="humarin/chatgpt_paraphraser_on_T5_base",
    #model_name="lvwerra/gpt2-imdb",
    #steps=51200,
    learning_rate=1.41e-5, remove_unused_columns=False, log_with="wandb",
    batch_size= 1024
)

txt_in_len = 5
txt_out_len = 20
seed = 1

In [10]:
np.random.seed(seed)

In [ ]:
#t5_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
t5_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
t5_model_ref = create_reference_model(t5_model)
t5_tokenizer = AutoTokenizer.from_pretrained(config.model_name, )

t5_tokenizer.pad_token = t5_tokenizer.eos_token

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
# create the dataset
#
dataset = load_dataset("imdb", split="train")
dataset = dataset.rename_columns({"text": "review", "label": "sentiment"})
# make sure the comments are are at least 500 and trim to 1000
dataset = dataset.filter(lambda x: 200 <= len(x["review"].split(" ")) <= 300, batched=False)
#dataset = dataset.map(lambda x: {"review": x["review"][:1000]}, batched=False)

dataset

In [ ]:
dataset = dataset.map(
    lambda x: {"input_ids": t5_tokenizer.encode('paraphrase: ' + x["review"], return_tensors="pt", max_length=300, truncation=True)[0,:]},
    batched=False,
)
dataset = dataset.map(lambda x: {"query": t5_tokenizer.decode(x["input_ids"])}, batched=False)
dataset = dataset[:20480]

from datasets import Dataset

dataset = Dataset.from_dict(dataset)
dataset.set_format("pytorch")

In [ ]:
dataset[3]["query"]

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [ ]:
ppo_trainer = PPOTrainer(config, t5_model, t5_model_ref, t5_tokenizer, dataset, data_collator=collator)

In [13]:
def pos_logit_to_reward(review: str, response: str):
    review_len_70 = round(len(review.split(" ")) * 0.7) ## 70% of review length
    response_len = len(response.split(" "))
    diff = abs(review_len_70 - response_len)
    reward = -math.log(max(diff,1e-8))
    return torch.tensor(reward)

In [ ]:
generation_kwargs = {
    "min_length": 1,
    "max_length": 350,
    #"max_new_tokens": 350,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    #"pad_token_id": t5_tokenizer.eos_token_id,
    "num_beams": 5,
    "temperature": 0.6,
    #"eos_token_id": -1,
}

In [15]:
%%time

average_lengths_ratio = []
for epoch in range(1):
    for batch in tqdm(ppo_trainer.dataloader):
        (logs, game_data,) = (
            dict(),
            dict(),
        )

        #### prepend a random control token
#         task_list = choices(ctrl_str, k=config.batch_size)
        game_data["query"] = batch["query"]
        query_tensors =  batch["input_ids"]

        #### get response from gpt2
        response_tensors = []
        for query in query_tensors:
            response = ppo_trainer.generate(query.squeeze(), **generation_kwargs)
            response_tensors.append(response.squeeze()[:350])
        game_data["response"] = [t5_tokenizer.decode(r.squeeze()) for r in response_tensors]

        #### generation length analysis

        length_ratios = [round(len(r.split(" ")) / len(q.split(" ")) ,3) for q, r in zip(batch["query"], game_data["response"])]
        average_lengths_ratio.append(sum(length_ratios) / len(length_ratios))

        print(f"Average length of the generated texts is: {average_lengths_ratio[-1]}")
        rewards = [pos_logit_to_reward(q,r) for q, r in zip(batch["query"], game_data["response"])]

        #### Run PPO training
        t = time.time()
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

#         for cs in ctrl_str:
#             key = "env/reward_" + cs.strip("[]")
#             stats[key] = np.mean([r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs])
        ppo_trainer.log_stats(stats, game_data, rewards)

  0%|          | 0/18 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Average length of the generated texts is: 0.25554296875
Average length of the generated texts is: 0.2843710937499999
Average length of the generated texts is: 0.43034375000000014
Average length of the generated texts is: 0.4218398437500001
Average length of the generated texts is: 0.45677343750000016
Average length of the generated texts is: 0.5218203124999998
Average length of the generated texts is: 0.49238671875
Average length of the generated texts is: 0.4727539062499999
Average length of the generated texts is: 0.4304414062499997


KeyboardInterrupt: ignored

In [ ]:
t5_model.save_pretrained("/content/drive/MyDrive/t5-paraphrase-len")
t5_tokenizer.save_pretrained("/content/drive/MyDrive/t5-paraphrase-len")

In [1]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

In [ ]:
%%time

average_lengths_ratio = []
for epoch in range(1):
    for batch in tqdm(ppo_trainer.dataloader):
        (logs, game_data,) = (
            dict(),
            dict(),
        )

        #### prepend a random control token
#         task_list = choices(ctrl_str, k=config.batch_size)
        game_data["query"] = batch["query"]
        query_tensors =  batch["input_ids"]

        #### get response from gpt2
        response_tensors = []
        for index, query in enumerate(query_tensors):
            response = ppo_trainer.generate(query.squeeze(), **generation_kwargs)
            response_tensors.append(response.squeeze()[:350])
            if index >= 1:
              break
        break

#         game_data["response"] = [t5_tokenizer.decode(r.squeeze()) for r in response_tensors]

#         #### generation length analysis

#         length_ratios = [round(len(r.split(" ")) / len(q.split(" ")) ,3) for q, r in zip(batch["query"], game_data["response"])]
#         average_lengths_ratio.append(sum(length_ratios) / len(length_ratios))

#         print(f"Average length of the generated texts is: {average_lengths_ratio[-1]}")
#         rewards = [pos_logit_to_reward(q,r) for q, r in zip(batch["query"], game_data["response"])]

#         #### Run PPO training
#         t = time.time()
#         stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

# #         for cs in ctrl_str:
# #             key = "env/reward_" + cs.strip("[]")
# #             stats[key] = np.mean([r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs])
#         ppo_trainer.log_stats(stats, game_data, rewards)